# Examen SPSI 2021 - 2022

**Autores**: 

- Ana Buendía Ruiz-Azuaga
- Andrés Millán Muñoz
- Paula Villanueva Núñez
- Juan Antonio Villegas Recio

## 📋 Ejercicio 3

> **Diseñe razonadamente el entorno de un ejemplo realista para el intercambio de una clave según el sistema de Diffie-Hellman y proceda al intercambio de una de ellas. Para llevar a cabo este ejemplo puede usar sagemath y openssl, pero el ejemplo que construya debe ser distinto en los datos a cualquiera que figure en los apuntes (p.e. Ejemplo 5.5.3 de la pág. 122).**

En una conexión SSH se establecen dos etapas: La primera es acordar una clave de cifrado para proteger la comunicación futura y la segunda es autenticar al usuario y comprobar si se le debe dar acceso al servidor. El esquema Diffie-Hellman es utilizado en la primera de las fases, donde ambas partes negocian una clave de sesión compartida, pero secreta. Al utilizar Diffie-Hellman, cada parte puede combinar sus propios datos privados con datos públicos del otro sistema para así llegar a dicha clave compartida. 

Esta clave de sesión se utiliza para cifrar toda la sesión. No combiene confundir las claves privadas utilizadas en este paso con las claves SSH utilizadas para autenticar un cliente en el servidor, ya que son conceptos distintos y están completamente separados.

El procedimiento seguido es el siguiente. Por simplicidad llamaremos $A$ y $B$ a las dos partes.

Utilizamos `openssl` para generar un primo seguro suficientemente grande, por ejemplo, de 64 bits:

```bash
$ openssl prime -generate -safe -bits 64
17885555072688303479
```

A partir de este número primo que llamaremos $n$, elegimos $g$, otro número primo que es además un elemento primitivo de $GF(n)$:

In [ ]:
n = 17885555072688303479
g = GF(n).primitive_element()

Al ser $g$ un elemento primitivo de $GF(n)$, tenemos asegurado que $1<g<n$. La pareja $(n,g)$ no tiene por qué ser secreta, de hecho puede ser compartida por más usuarios.

In [ ]:
print(f"(n,g) = {(n,g)}")

### A elige su clave secreta y calcula X
A continuación A elige aleatoriamente un número elevado $x$ y calcula el número $X=g^x \mod n$, el cual será enviado a B.

In [ ]:
# A

x = randint(0,2**20)
X = g**x % n

print(f"La clave secreta de A es x = {x}")
print(f"A le enviará a B el número X = {X}")

### B elige su clave secreta y calcula Y
Del mismo modo, B elige aleatoriamente un número elevado $y$ y calcula el número $Y=g^y \mod n$, el cual será enviado a A.

In [ ]:
# B

y = randint(0,2**20)
Y = g**y % n

print(f"La clave secreta de B es y = {y}")
print(f"B le enviará a A el número Y = {Y}")

### Intercambio
A y B se intercambian $X$ e $Y$, manteniendo secretos los exponentes $x$ e $y$. A recibe $Y$ y calcula $k_1$:

In [ ]:
# A

k1 = Y**x % n

print(f"Clave calculada por A: {k1}")

B recibe $X$ y calcula $k_2$

In [ ]:
# B

k2 = X**y % n

print("Clave calculada por B: {k2}")

Comprobamos que en efecto $k_1$ y $k_2$ coinciden, siendo calculadas independientemente y sin revelar las claves secretas $x$ e $y$. Este número $k_1=k_2$ es la clave secreta compartida entre A y B, que estas utilizarán posteriormente para cifrar la comunicación en SSH.

In [ ]:
k1 == k2

Este proceso permite a cada parte participar igualmente en la generación del secreto compartido, lo que no permite que un extremo controle el secreto. También cumple la tarea de generar un secreto compartido idéntico sin tener que enviar esa información a través de canales inseguros. La clave generada es simétrica, lo que significa que la misma clave utilizada para cifrar un mensaje se puede utilizar para descifrarlo en el otro lado. El propósito de esto es envolver toda la comunicación adicional en un túnel encriptado que no pueda ser descifrado por personas externas.

## 📋 Ejercicio 4

> **Escenifique compartir el secreto de valor 113132 entre 50 partícipes, requiriéndose el acuerdo de 42 de ellos para explicitar dicho secreto. Use en este ejercicio el esquema de Shamir de intercambio de secretos, pudiendo el alumno servirse de Sagemath implementando su propio software**

Como se nos pide, vamos a ejemplificar el esquema de Shamir. 

Para empezar, escribamos nuestras hipótesis, usando la nomenclatura de la teoría:

In [ ]:
# Secreto
s = 113132 

# Número de participantes
n = 50

# Umbral de participantes para recuperar el secreto
t = 42

Para operar, debemos realizar los siguientes pasos:

1. Se elige un número primo $p$ tal que $p > máx\{n, s\}$
2. Se eligen $t - 1$ elementos de $Z_p$, $a_1, \dotsc, a_{t-1}$ de forma que $a_{t-1} \ne 0$
3. Consideramos el polinomio $p(x) = s + a_1x + \dotsc + a_{t-1}x^{t-1}$.
4. Son escogidos $x_1, \dotsc, x_n \in Z_p^*$, y se calcula para todo $1 \le i \le n$ el valor $y_i = p(x_i)$
5. El partícipe $i, 1 \le i \le n$, recibe la parte del secreto $<x_i, y_i>$ y la mantendrá como par secreto

Sin más dilación, vamos a ello:

In [ ]:
# ──────────────────────────────────────────────────────────────────────── 1 ─────

p = random_prime(2**64)

while p <= max(n, s):
    p = random_prime(2**64)

print(f'El primo generado es p = {p}. \n\t -> ¿Es mayor que {n} y {s}? {p > max(n, s)}\n')


# Definimos los anillos necesarios para operar
Zp = GF(p)
K.<x> = PolynomialRing(Zp)



# ──────────────────────────────────────────────────────────────────────── 2 ─────

A = [Zp.random_element() for i in range(0, t-1)]
print(f'Para A, hemos escogido {len(A)} = t-1 = {t-1} elementos de Zp\n')


# Nos aseguramos que A[t-2] = a_{t-1} != 0
while (A[t-2] == 0):
    A[t-2] = Zp.random_element()
    
print(f'Se tiene además que A[t-2] = a_(t-1) = {A[-1]} != 0\n')



# ──────────────────────────────────────────────────────────────────────── 3 ─────

p = K(A)*x + s
print(f'Escogemos el polinomio \n{p}\n\n')



# ──────────────────────────────────────────────────────────────────────── 4 ─────

X = [Zp.random_element() for i in range(0, n)]

# Limpiamos los posibles 0 de X
while True:
    try:
        i = X.index(0)
        X[i] = Zp.random_element()
    except ValueError:
        # Si no hay 0s, paramos
        break
    
Y = [p(x) for x in X]



# ──────────────────────────────────────────────────────────────────────── 5 ─────

pares_secretos = [ (x,y) for x, y in zip(X,Y) ]

print(f'Hemos obtenido {len(pares_secretos)} pares secretos')

Mostremos ahora la recuperación del secreto, escogemos $t$ participantes al azar:

In [ ]:
import random 
random.shuffle(pares_secretos)

estan_de_acuerdo = pares_secretos[:t]

print(f'Estos son los pares de las {t} personas que están de acuerdo:')

estan_de_acuerdo

Y ahora, estas personas que están de acuerdo, llevan a cabo la interpolación de Lagrange. Podría usarse el de Newton, pero teniendo en cuenta que el de Lagrange se ejecuta lo suficientemente rápido, lo escogeremos para esta prueba. Sage proporciona una manera rápida de obtenerlo que nos resulta ventajosa.

De esta forma, se descubre el secreto:

In [ ]:
q = K.lagrange_polynomial(estan_de_acuerdo)
q(0)

¿Qué ocurriría si no se llegara a la cantidad de personas requerida? 

In [ ]:
random.shuffle(pares_secretos)

frustrados = pares_secretos[:randint(1, t-1)]
r = K.lagrange_polynomial(frustrados)
r(0)

No son capaces de conseguir el secreto.